# Adding custom tools to your Strands Agents / 为您的 Strands Agents 添加自定义工具

## Overview / 概述
In this example we will guide you through the different ways to create custom tools using Strands Agents. We will build a personal assistant use case that connects with a local SQLite database to perform data tasks. As a bonus, we will also guide you through the usage of the reasoning capabilities on Claude Sonnet 3.7 using the `thinking` field of the `BedrockModel` class

在此示例中，我们将指导您使用 Strands Agents 创建自定义工具的不同方法。我们将构建一个个人助手用例，连接到本地 SQLite 数据库来执行数据任务。作为额外奖励，我们还将指导您使用 `BedrockModel` 类的 `thinking` 字段在 Claude Sonnet 3.7 上使用推理功能

## Agent Details / 代理详情
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Native tools used   |current_time, calculator                           |
|Custom tools created|create_appointment, list_appointments              |
|Agent Structure     |Single agent architecture                          |

</div>


## Architecture / 架构

<div style="text-align:left">
    <img src="images/architecture.png" width="85%" />
</div>

## Key Features / 关键特性
* **单代理架构**：此示例创建一个与内置和自定义工具交互的单个代理
* **内置工具**：学习如何使用 Strands Agent 的工具
* **自定义工具**：学习如何创建您自己的工具
* **Bedrock 模型作为底层 LLM**：使用来自 Amazon Bedrock 的 Anthropic Claude 3.7 作为底层 LLM 模型

### Importing dependency packages / 导入依赖包

Now let's import the dependency packages

现在让我们导入依赖包

In [ ]:
import json
import sqlite3
import uuid
from datetime import datetime

from strands import Agent, tool
from strands.models import BedrockModel

## Defining custom tools / 定义自定义工具
Next let's define custom tools to interact with a local SQLite database:
* **create_appointment**: create a new personal appointment with unique id, date, location, title and description 
* **list_appointment**: list all available appointments
* **update_appointments**: update an appointment based on the appointment id

接下来让我们定义与本地 SQLite 数据库交互的自定义工具：
* **create_appointment**：创建一个具有唯一 ID、日期、位置、标题和描述的新个人约会
* **list_appointment**：列出所有可用的约会
* **update_appointments**：根据约会 ID 更新约会

### Defining tools in the same file of your agent / 在代理的同一文件中定义工具

There are multiple ways to define tools with the Strands Agents SDK. The first one is to add a `@tool` decorator to your function and provide the documentation to it. In this case, Strands Agents will use the function documentation, typing and arguments to provide the tools to your agent. In this case, you can even define the tool in the same file as your agent

使用 Strands Agents SDK 定义工具有多种方法。第一种是在函数中添加 `@tool` 装饰器并为其提供文档。在这种情况下，Strands Agents 将使用函数文档、类型和参数来为您的代理提供工具。在这种情况下，您甚至可以在与代理相同的文件中定义工具

In [ ]:
@tool
def create_appointment(date: str, location: str, title: str, description: str) -> str:
    """
    Create a new personal appointment in the database.

    Args:
        date (str): Date and time of the appointment (format: YYYY-MM-DD HH:MM).
        location (str): Location of the appointment.
        title (str): Title of the appointment.
        description (str): Description of the appointment.

    Returns:
        str: The ID of the newly created appointment.

    Raises:
        ValueError: If the date format is invalid.
    """
    # Validate date format
    try:
        datetime.strptime(date, "%Y-%m-%d %H:%M")
    except ValueError:
        raise ValueError("Date must be in format 'YYYY-MM-DD HH:MM'")

    # Generate a unique ID
    appointment_id = str(uuid.uuid4())

    conn = sqlite3.connect("appointments.db")
    cursor = conn.cursor()

    # Create the appointments table if it doesn't exist
    cursor.execute(
        """
    CREATE TABLE IF NOT EXISTS appointments (
        id TEXT PRIMARY KEY,
        date TEXT,
        location TEXT,
        title TEXT,
        description TEXT
    )
    """
    )

    cursor.execute(
        "INSERT INTO appointments (id, date, location, title, description) VALUES (?, ?, ?, ?, ?)",
        (appointment_id, date, location, title, description),
    )

    conn.commit()
    conn.close()
    return f"Appointment with id {appointment_id} created"

### Tool definition with Module-Based Approach / 使用基于模块的方法定义工具

You can also define your tools as a standalone file and import it to your agent. In this case you can still use the decorator approach or you could also define your function using a TOOL_SPEC dictionary. The formating is similar to the one used by the [Amazon Bedrock Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use-examples.html) for tool usage. In this case you are more flexible to define the required parameters as well as the return of success and error executions and TOOL_SPEC definitions will work in this case.

您也可以将工具定义为独立文件并将其导入到您的代理中。在这种情况下，您仍然可以使用装饰器方法，或者也可以使用 TOOL_SPEC 字典定义函数。格式类似于 [Amazon Bedrock Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use-examples.html) 用于工具使用的格式。在这种情况下，您在定义所需参数以及成功和错误执行的返回方面更加灵活，TOOL_SPEC 定义在这种情况下将有效。

#### Decorator approach / 装饰器方法

When defining your tool using a decorator in a standalone file, your process is very similar to the one in the same file as your agent, but you will need to import or agent tool later on.

在独立文件中使用装饰器定义工具时，您的过程与在代理相同文件中的过程非常相似，但您需要稍后导入或代理工具。

In [ ]:
%%writefile list_appointments.py
import json
import sqlite3
import os
from strands import tool

@tool
def list_appointments() -> str:
    """
    List all available appointments from the database.
    
    Returns:
        str: the appointments available 
    """
    # Check if database exists
    if not os.path.exists('appointments.db'):
        return "No appointment available"
    
    conn = sqlite3.connect('appointments.db')
    conn.row_factory = sqlite3.Row  # This enables column access by name
    cursor = conn.cursor()
    
    # Check if the appointments table exists
    try:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='appointments'")
        if not cursor.fetchone():
            conn.close()
            return "No appointment available"
        
        cursor.execute("SELECT * FROM appointments ORDER BY date")
        rows = cursor.fetchall()
        
        # Convert rows to dictionaries
        appointments = []
        for row in rows:
            appointment = {
                'id': row['id'],
                'date': row['date'],
                'location': row['location'],
                'title': row['title'],
                'description': row['description']
            }
            appointments.append(appointment)
        
        conn.close()
        return json.dumps(appointments)
    
    except sqlite3.Error:
        conn.close()
        return []


#### TOOL_SPEC approach / TOOL_SPEC 方法

Alternativelly, you can use the TOOL_SPEC approach when defining your tool

或者，您可以在定义工具时使用 TOOL_SPEC 方法

In [ ]:
%%writefile update_appointment.py
import sqlite3
from datetime import datetime
import os
from strands.types.tools import ToolResult, ToolUse
from typing import Any

TOOL_SPEC = {
    "name": "update_appointment",
    "description": "Update an appointment based on the appointment ID.",
    "inputSchema": {
        "json": {
            "type": "object",
            "properties": {
                "appointment_id": {
                    "type": "string",
                    "description": "The appointment id."
                },
                "date": {
                    "type": "string",
                    "description": "Date and time of the appointment (format: YYYY-MM-DD HH:MM)."
                },
                "location": {
                    "type": "string",
                    "description": "Location of the appointment."
                },
                "title": {
                    "type": "string",
                    "description": "Title of the appointment."
                },
                "description": {
                    "type": "string",
                    "description": "Description of the appointment."
                }
            },
            "required": ["appointment_id"]
        }
    }
}
# Function name must match tool name
def update_appointment(tool: ToolUse, **kwargs: Any) -> ToolResult:
    tool_use_id = tool["toolUseId"]
    appointment_id = tool["input"]["appointment_id"]
    if "date" in tool["input"]:
        date = tool["input"]["date"]
    else:
        date = None
    if "location" in tool["input"]:
        location = tool["input"]["location"]
    else:
        location = None
    if "title" in tool["input"]:
        title = tool["input"]["title"]
    else:
        title = None
    if "description" in tool["input"]:
        description = tool["input"]["description"]
    else:
        description = None
        
    # Check if database exists
    if not os.path.exists('appointments.db'): 
        return {
            "toolUseId": tool_use_id,
            "status": "error",
            "content": [{"text": f"Appointment {appointment_id} does not exist"}]
        } 
    
    # Check if appointment exists
    conn = sqlite3.connect('appointments.db')
    cursor = conn.cursor()
    
    # Check if the appointments table exists
    try:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='appointments'")
        if not cursor.fetchone():
            conn.close()
            return {
                "toolUseId": tool_use_id,
                "status": "error",
                "content": [{"text": f"Appointments table does not exist"}]
            }
        
        cursor.execute("SELECT * FROM appointments WHERE id = ?", (appointment_id,))
        appointment = cursor.fetchone()
        
        if not appointment:
            conn.close()
            return {
                "toolUseId": tool_use_id,
                "status": "error",
                "content": [{"text": f"Appointment {appointment_id} does not exist"}]
            }
        
        # Validate date format if provided
        if date:
            try:
                datetime.strptime(date, '%Y-%m-%d %H:%M')
            except ValueError:
                conn.close()
                return {
                    "toolUseId": tool_use_id,
                    "status": "error",
                    "content": [{"text": "Date must be in format 'YYYY-MM-DD HH:MM'"}]
                }
        
        # Build update query
        update_fields = []
        params = []
        
        if date:
            update_fields.append("date = ?")
            params.append(date)
        
        if location:
            update_fields.append("location = ?")
            params.append(location)
        
        if title:
            update_fields.append("title = ?")
            params.append(title)
        
        if description:
            update_fields.append("description = ?")
            params.append(description)
        
        # If no fields to update
        if not update_fields:
            conn.close()
            return {
                "toolUseId": tool_use_id,
                "status": "success",
                "content": [{"text": "No need to update your appointment, you are all set!"}]
            }
        
        # Complete the query
        query = f"UPDATE appointments SET {', '.join(update_fields)} WHERE id = ?"
        params.append(appointment_id)
        
        cursor.execute(query, params)
        conn.commit()
        conn.close()
        
        return {
            "toolUseId": tool_use_id,
            "status": "success",
            "content": [{"text": f"Appointment {appointment_id} updated with success"}]
        }
    
    except sqlite3.Error as e:
        conn.close()
        return {
            "toolUseId": tool_use_id,
            "status": "error",
            "content": [{"text": str(e)}]
        }


let's now import `list_appointments` and `update_appointment` as a tool

现在让我们将 `list_appointments` 和 `update_appointment` 作为工具导入

In [ ]:
import list_appointments
import update_appointment

## Creating Agent / 创建代理

Now that we have created our custom tools, let's define our first agent. To do so, we need to create a system prompt that defines what the agent should and should not do. We will then define our agent's underlying LLM model and we will provide it with built-in and custom tools. 

现在我们已经创建了自定义工具，让我们定义我们的第一个代理。为此，我们需要创建一个系统提示，定义代理应该做什么和不应该做什么。然后我们将定义代理的底层 LLM 模型，并为其提供内置和自定义工具。

#### Setting agent system prompt / 设置代理系统提示
In the system prompt we will define the instructions for our agent

在系统提示中，我们将为代理定义指令

In [ ]:
system_prompt = """You are a helpful personal assistant that specializes in managing my appointments and calendar. 
You have access to appointment management tools, a calculator, and can check the current time to help me organize my schedule effectively. 
Always provide the appointment id so that I can update it if required"""

#### Defining agent underlying LLM model / 定义代理底层 LLM 模型

Next let's define our agent underlying model. Strands Agents natively integrate with Amazon Bedrock models, and provides the ability to configure how the model is called. Below, you can see a simple initialization of a `BedrockModel` provider, with some of the optional configurations commented out. You can learn more about configuration options, and default values, at [Strands Agents Bedrock Model Provider documentation](https://strandsagents.com/0.1.x/user-guide/concepts/model-providers/amazon-bedrock/). For our example, we will use the `Anthropic Claude 3.7 Sonnet` model from Bedrock.

接下来让我们定义代理的底层模型。Strands Agents 原生集成 Amazon Bedrock 模型，并提供配置模型调用方式的能力。下面，您可以看到 `BedrockModel` 提供商的简单初始化，其中一些可选配置被注释掉了。您可以在 [Strands Agents Bedrock 模型提供商文档](https://strandsagents.com/0.1.x/user-guide/concepts/model-providers/amazon-bedrock/) 中了解更多关于配置选项和默认值的信息。在我们的示例中，我们将使用来自 Bedrock 的 `Anthropic Claude 3.7 Sonnet` 模型。

In [ ]:
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    # region_name="us-east-1",
    # boto_client_config=Config(
    #    read_timeout=900,
    #    connect_timeout=900,
    #    retries=dict(max_attempts=3, mode="adaptive"),
    # ),
    # temperature=0.9,
    # max_tokens=2048,
)

#### Import built-in tools / 导入内置工具

The next step to build our agent is to import our Strands Agents built-in tools. Strands Agents provides a set of commonly used built-in tools in the optional package `strands-tools`. You have tools for RAG, memory, file operations, code interpretation and others available in this repo. For our example we will use the `current_time` tool to provide our agent with the information about the current time and the `calculator` tool to do some math

构建代理的下一步是导入 Strands Agents 内置工具。Strands Agents 在可选包 `strands-tools` 中提供了一组常用的内置工具。您可以在此仓库中找到 RAG、内存、文件操作、代码解释等工具。在我们的示例中，我们将使用 `current_time` 工具为代理提供当前时间信息，使用 `calculator` 工具进行一些数学计算

In [ ]:
from strands_tools import calculator, current_time

#### Defining Agent / 定义代理

Now that we have all the required information available, let's define our agent

现在我们已经有了所有所需的信息，让我们定义我们的代理

In [ ]:
agent = Agent(
    model=model,
    system_prompt=system_prompt,
    tools=[
        current_time,
        calculator,
        create_appointment,
        list_appointments,
        update_appointment,
    ],
)

## Invoking agent / 调用代理

Let's now invoke our restaurant agent with a greeting and analyse its results

现在让我们用问候调用我们的餐厅代理并分析其结果

In [ ]:
results = agent("How much is 2+2?")

#### Analysing the agent's results / 分析代理的结果

Nice! We've invoked our agent for the first time! Let's now explore the results object. First thing we can see is the messages being exchanged by the agent in the agent's object

太好了！我们第一次调用了我们的代理！现在让我们探索结果对象。我们可以看到的第一件事是代理在代理对象中交换的消息

In [ ]:
agent.messages

Next we can take a look at the usage of our agent for the last query by analysing the result `metrics`

接下来，我们可以通过分析结果 `metrics` 来查看代理在最后一次查询中的使用情况

In [ ]:
results.metrics

#### Invoking agent with follow up question / 使用后续问题调用代理
Ok, let's now make an appointment for tomorrow

好的，现在让我们为明天安排一个约会

In [ ]:
results = agent(
    "Book 'Agent fun' for tomorrow 3pm in NYC. This meeting will discuss all the fun things that an agent can do"
)

#### Updating appointment / 更新约会

Let's now update this appointment

现在让我们更新这个约会

In [ ]:
results = agent("Oh no! My bad, 'Agent fun' is actually happening in DC")

#### Analysing the agent's results / 分析代理的结果
Let's look at the agent messages and result metrics again

让我们再次查看代理消息和结果指标

In [ ]:
agent.messages

In [ ]:
results.metrics

#### Checking tool usage from messages / 从消息中检查工具使用情况

Let's deep-dive into the tool usage in the messages dictionary. Later on we will show case how to observe and evaluate your agent's behavior, but this is the first step in this direction

让我们深入研究消息字典中的工具使用情况。稍后我们将展示如何观察和评估您的代理行为，但这是这个方向的第一步

In [ ]:
for m in agent.messages:
    for content in m["content"]:
        if "toolUse" in content:
            print("Tool Use:")
            tool_use = content["toolUse"]
            print("\tToolUseId: ", tool_use["toolUseId"])
            print("\tname: ", tool_use["name"])
            print("\tinput: ", tool_use["input"])
        if "toolResult" in content:
            print("Tool Result:")
            tool_result = m["content"][0]["toolResult"]
            print("\tToolUseId: ", tool_result["toolUseId"])
            print("\tStatus: ", tool_result["status"])
            print("\tContent: ", tool_result["content"])
            print("=======================")

### Validating that the action was performed correctly / 验证操作是否正确执行
Let's now check our database to confirm that the operations where done correctly. The `Agent` class has the ability for directly calling tools the agent was initialized with by calling `agent.tool.<tool_name>(<tool_params>)`. Direct tool calls are great for giving the agent information from a tool without needing the agent to invoke that tool itself. We can use this direct tool invocation to list the current appointments:

现在让我们检查数据库以确认操作是否正确完成。`Agent` 类具有通过调用 `agent.tool.<tool_name>(<tool_params>)` 直接调用代理初始化时的工具的能力。直接工具调用非常适合从工具给代理提供信息，而无需代理自己调用该工具。我们可以使用这种直接工具调用来列出当前约会：

In [ ]:
list_appointments_result = agent.tool.list_appointments()
print(json.dumps(list_appointments_result, indent=2))

We can see that the result of executing the tool is in the ToolResult format, including a `toolUseId`, an execution `status`, and the `content` of the response. We can better visualize the tool's result like this:

我们可以看到执行工具的结果采用 ToolResult 格式，包括 `toolUseId`、执行 `status` 和响应的 `content`。我们可以这样更好地可视化工具的结果：

In [ ]:
list_appointments_result_text_content = list_appointments_result["content"][0]["text"]
print(json.dumps(json.loads(list_appointments_result_text_content), indent=2))

Finally, when executing tools using direct tool invocation, the agent records these executions in its messages history. By default this is enabled, but can be disabled with the `record_direct_tool_call` boolean flag attribute on the `Agent` class.

最后，当使用直接工具调用执行工具时，代理会在其消息历史中记录这些执行。默认情况下这是启用的，但可以通过 `Agent` 类上的 `record_direct_tool_call` 布尔标志属性禁用。

In [ ]:
current_time_result = agent.tool.current_time()
print("Current Time direct tool call result:")
print(current_time_result)
current_time_direct_tool_messages = agent.messages[-4:]
print("Current Time direct tool call messages:")
print(current_time_direct_tool_messages)

agent.record_direct_tool_call = False # Set the record_direct_tool_call to False
agent.tool.list_appointments()
after_disable_record_messages = agent.messages[-4:]
print("After disabling record direct tool call messages, history should not have changed:")
print(current_time_direct_tool_messages == after_disable_record_messages)

## Extension: Extended Thinking / 扩展：扩展思维
 
Extended thinking enables supported Claude-family models the ability to leverage enhanced reasoning capabilities for complex tasks, providing transparent step-by-step thought processes before delivering final answers. To enable thinking, you can include the configuration below when configuring your Bedrock ModelProvider. You can learn more at [AWS's documentation on Extended Thinking](https://docs.aws.amazon.com/bedrock/latest/userguide/claude-messages-extended-thinking.html).

扩展思维使支持的 Claude 系列模型能够利用增强的推理能力来处理复杂任务，在提供最终答案之前提供透明的逐步思维过程。要启用思维，您可以在配置 Bedrock ModelProvider 时包含以下配置。您可以在 [AWS 关于扩展思维的文档](https://docs.aws.amazon.com/bedrock/latest/userguide/claude-messages-extended-thinking.html) 中了解更多信息。

In [ ]:
thinking_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    additional_request_fields={
        "thinking": {
            "type": "enabled",
            "budget_tokens": 2048,
        }
    },
)

After defining the `thinking_model`, you can create and invoke a new `thinking_agent`:

定义 `thinking_model` 后，您可以创建并调用新的 `thinking_agent`：

In [ ]:
thinking_system_prompt = """You are a helpful personal assistant that specializes in managing my appointments and calendar. 
You have access to appointment management tools, a calculator, and can check the current time to help me organize my schedule effectively. 
You think through your problem, step by step, to come up with an answer.
Always provide the appointment id so that I can update it if required"""

thinking_agent = Agent(
    model=thinking_model,
    system_prompt=thinking_system_prompt,
    tools=[
        current_time,
        calculator,
        create_appointment,
        list_appointments,
        update_appointment,
    ],
)

thinking_result = thinking_agent("I want to add a new appointment for tomorrow at 2pm")

We can better analyze the extended thinking capabilities by printing out the agent's messages. Extended thinking is represented as `reasoningContent` blocks in the response from the agent.

我们可以通过打印代理的消息来更好地分析扩展思维能力。扩展思维在代理的响应中表示为 `reasoningContent` 块。

In [ ]:
thinking_agent.messages

## Great work ! / 做得好！
